## Function Calling

In [ ]:
import os

import azure.identity
import openai
import rich
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv(override=True)
API_HOST = os.getenv("API_HOST", "github")

client = openai.OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"],
)

MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

In [ ]:
class GetDeliveryDate(BaseModel):
    order_id: str

In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "You're a customer support bot. Use the tools to assist the user."},
        {"role": "user", "content": "Hi, can you tell me the delivery date for my order #12345?"},
    ],
    tools=[openai.pydantic_function_tool(GetDeliveryDate)],
)

rich.print(response.choices[0].message.tool_calls[0].function)

Function(arguments='{"order_id":"12345"}', name='GetDeliveryDate')